In [1]:
import warnings

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score

import funciones_auxiliares as aux

# warnings.filterwarnings('ignore')
sns.set_theme()
plt.rcParams['figure.figsize'] = (12, 6)

In [18]:
df = pd.read_csv('income-db.csv')

In [22]:
df_depurada = df.replace('?', np.nan)
df_depurada = df_depurada.dropna()

In [25]:
def preprocesar_occupation(df):
    tmp = df.copy()

    dict_reemplazo = {'white-collar': ['Prof-specialty', 'Exec-managerial', 'Adm-clerical', 'Sales',
                                       'Tech-support'],
                      'blue-collar': ['Craft-repair', 'Machine-op-inspct', 'Transport-moving',
                                      'Handlers-cleaners', 'Farming-fishing', 'Protective-serv', 
                                      'Priv-house-serv'],
                      'others': ['Other-service', 'Armed-Forces']}

    for valor_nuevo, valores_antiguos in dict_reemplazo.items():
        tmp = tmp.replace(valores_antiguos, valor_nuevo)
    
    return tmp

In [26]:
limpia = preprocesar_occupation(df_depurada)

In [29]:
limpia.occupation.value_counts()

white-collar    24360
blue-collar     16040
others           4822
Name: occupation, dtype: int64